In [1]:
import os
import sys
import datetime
from pathlib import Path
import logging
from main import SigmoidFitter, MeltomeAtlasHandler, setup_logging

import json
import numpy as np
import pandas as pd
import seaborn as sns
# for logfile in [f for f in os.listdir() if '.log' in f]:
#     print(logfile)
    # os.remove(logfile)
now = datetime.datetime.now()
timestamp_str = now.strftime("%Y-%m-%d_%H-%M")
setup_logging(f'./{timestamp_str}_testing.log', logging.DEBUG)

2025-12-10 12:35:33 - root - INFO - Logging to file: ./2025-12-10_12-35_testing.log
2025-12-10 12:35:33 - root - INFO - Logging level: 10


In [2]:
# SCRIPT_DIR = Path(__file__).resolve().parent
CWD = Path(os.getcwd()).resolve()
PROJECT_ROOT = CWD.parent.parent
print(PROJECT_ROOT)
file_path = PROJECT_ROOT / "Data" / "FLIP" / "splits" / "meltome" / "full_dataset.json"
output_path = f'./results_meltome_{timestamp_str}'
print(file_path)

C:\Users\alexa\Documents\PROHITS
C:\Users\alexa\Documents\PROHITS\Data\FLIP\splits\meltome\full_dataset.json


In [3]:
# Sample subset of Meltome
data_handler = MeltomeAtlasHandler(file_path, output_path, logging.DEBUG)
print(data_handler.data.shape)
df = data_handler.select_subset(10)
chunk0 = data_handler.data.iloc[:25]
data_handler.data = df 
print(data_handler.data.shape)

2025-12-10 12:35:33 - MeltomeAtlasHandler - INFO - Meltome Handler initialization
2025-12-10 12:35:33 - MeltomeAtlasHandler - DEBUG - DEBUG Mode
2025-12-10 12:35:35 - MeltomeAtlasHandler - INFO - Meltome data loaded from C:\Users\alexa\Documents\PROHITS\Data\FLIP\splits\meltome\full_dataset.json
2025-12-10 12:35:35 - MeltomeAtlasHandler - INFO - Meltome header ['proteinId', 'uniprotAccession', 'runName', 'meltingPoint', 'meltingBehaviour', 'sequence', 'quantNormMeltingPoint']
2025-12-10 12:35:35 - MeltomeAtlasHandler - INFO - Output directory: c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\results_meltome_2025-12-10_12-35
(221203, 7)
(250, 7)


In [4]:
n = np.arange(20)
s = np.array_split(n, 10)
print(data_handler.data.shape[0], len(s), np.sum([len(i) for i in s]))

250 10 20


In [5]:
test_data = pd.DataFrame(data_handler.data.iloc[0].meltingBehaviour)

In [6]:
test_data

,temperature,channel,fold_change
0,25.0,TMT126,1.000000
1,36.0,TMT127H,0.838219
2,30.0,TMT127L,0.950526
3,45.0,TMT128H,0.571089
4,39.0,TMT128L,0.722860
5,56.0,TMT129H,0.205821
6,50.0,TMT129L,0.415258
7,66.9,TMT130H,0.206312
8,59.6,TMT130L,0.217382
9,70.4,TMT131L,0.188792


In [7]:
t = SigmoidFitter()
print(t.get_statistics())
r = t.fit_curve(test_data.temperature.to_numpy(), test_data.fold_change.to_numpy())
print(t.get_statistics())

{'rmse': nan, 'r_squared': nan}
{'rmse': np.float64(0.03133592280233545), 'r_squared': np.float64(0.9898641838335815)}


In [8]:
x = t.get_melting_temp()
print(x)

45.94


c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: divide by zero encountered in divide
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


In [9]:
results0 = data_handler.process_chunk(chunk0)

2025-12-10 12:35:35 - MeltomeAtlasHandler - INFO - Chunk progress : 0 / 25, pid : A0A023PXQ4, specie : Saccharomyces cerevisiae lysate
2025-12-10 12:35:35 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 12:35:35 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9918, RMSE = 0.0280
2025-12-10 12:35:35 - SigmoidFitter - DEBUG - Fitted parameters: pl = -0.387047, a = 396.281525, b = 6.904978
2025-12-10 12:35:35 - MeltomeAtlasHandler - INFO - Chunk progress : 1 / 25, pid : A0A023T4K3, specie : Caenorhabditis elegans lysate
2025-12-10 12:35:35 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 12:35:35 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9867, RMSE = 0.0453
2025-12-10 12:35:35 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.052359, a = 719.635395, b = 19.066874
2025-12-10 12:35:35 - MeltomeAtlasHandler - INFO - Chunk progress : 2 / 25, pid : A0A023T778, specie : Mus musculus BMDC lysate
2025-12-10 12:35:35 - SigmoidFitter - DEBU

c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: overflow encountered in exp
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


In [10]:
results0_df = pd.DataFrame(results0)

In [11]:
data_handler.data

,proteinId,uniprotAccession,runName,meltingPoint,meltingBehaviour,sequence,quantNormMeltingPoint
37809,Q9ZV56_CCT1,Q9ZV56,Arabidopsis thaliana seedling lysate,45.937513,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MSNVIGDRTEDGLSTAAAASGSTAVQSSPPTDRPVRVYADGIYDLF...,NaN
4046,F4IIM1_CSI1,F4IIM1,Arabidopsis thaliana seedling lysate,39.295154,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MTSALGWRFPSTNGNGLAPSDTERNGDMKMHDSEPPTPHSTTKMSL...,NaN
7508,O80653_SKIP,O80653,Arabidopsis thaliana seedling lysate,NaN,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MKSLNDLPAPKSTTTTYYDHSNDAWFKNRVTESETVKSSSIKFKVV...,NaN
32685,Q9C8E6_PMI1,Q9C8E6,Arabidopsis thaliana seedling lysate,NaN,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MAGEYSGSRSSNTQLLAELEALSENLYQKPQVSVGNRRTNSLALPR...,NaN
32710,Q9CA28_TKPR2,Q9CA28,Arabidopsis thaliana seedling lysate,NaN,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MSEYLVTGGTGFIASYIIKSLLELGHTVRTTVRNPRDEEKVGFLWE...,NaN
...,...,...,...,...,...,...,...
64731,CAMSAP1,Q5T5Y3,pTcells,49.235472,"[{'temperature': 37.0, 'fold_change': 0.999985...",MVDASGRAAAEGWRKMEAPPDGAADLVPLDRYDAARAKIAANLQWI...,49.627580
167295,RAB2A,P61019,pTcells,58.300156,"[{'temperature': 37.0, 'fold_change': 0.999985...",MAYAYLFKYIIIGDTGVGKSCLLLQFTDKRFQPVHDLTIGVEFGAR...,59.843182
89384,EIF3G,O75821,pTcells,51.097380,"[{'temperature': 37.0, 'fold_change': 0.999985...",MPTGDFDSKPSWADQVEEEGEDDKCVTSELLKGIPLATGDTSPEPE...,50.745437
153550,PHB2,Q99623,pTcells,53.301503,"[{'temperature': 37.0, 'fold_change': 0.999985...",MAQNLKDLAGRLPAGPRGMGTALKLLLGAGAVAYGVRESVFTVEGG...,51.872882


In [12]:
results0_df

,pid,runName,pl,a,b,rmse,r_squared,tm_pred,tm_flip
0,A0A023PXQ4,Saccharomyces cerevisiae lysate,-0.387047,396.281525,6.904978,0.0280,0.9918,52.99,52.403034
1,A0A023T4K3,Caenorhabditis elegans lysate,0.052359,719.635395,19.066874,0.0453,0.9867,37.96,37.962947
2,A0A023T778,Mus musculus BMDC lysate,0.106919,1749.919673,32.061066,0.0564,0.9812,54.99,54.425342
3,A0A061ACF5,Caenorhabditis elegans lysate,0.278508,295.604929,6.790961,0.0246,0.9868,49.46,49.459216
4,A0A061ACH4,Caenorhabditis elegans lysate,0.023551,724.433779,17.056474,0.0390,0.9914,42.59,42.593131
5,A0A061ACH8,Caenorhabditis elegans lysate,0.052800,559.097425,14.824896,0.0189,0.9977,38.00,37.999478
6,A0A061ACH9,Caenorhabditis elegans lysate,0.022312,602.364195,14.213085,0.0412,0.9878,42.52,42.517527
7,A0A061ACI3,Caenorhabditis elegans lysate,0.237867,660.532837,15.694724,0.0486,0.9727,43.89,43.892211
8,A0A061ACL3,Caenorhabditis elegans lysate,0.081819,934.223132,22.436277,0.0397,0.9909,41.97,41.973245
9,A0A061ACL6,Caenorhabditis elegans lysate,-1152.945257,224.780039,-3.236776,0.0587,0.9484,49.87,49.044775


In [13]:
results = data_handler.process(10)

2025-12-10 12:35:41 - MeltomeAtlasHandler - INFO - Starting curve fitting process
2025-12-10 12:35:41 - MeltomeAtlasHandler - DEBUG - Data split into 10 chunks
2025-12-10 12:35:41 - MeltomeAtlasHandler - INFO - Processing chunk 0 / 10 (size : 25)
2025-12-10 12:35:41 - MeltomeAtlasHandler - INFO - Chunk progress : 0 / 25, pid : Q9ZV56, specie : Arabidopsis thaliana seedling lysate
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9899, RMSE = 0.0313
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.093248, a = 294.413366, b = 6.615403
2025-12-10 12:35:41 - MeltomeAtlasHandler - INFO - Chunk progress : 1 / 25, pid : F4IIM1, specie : Arabidopsis thaliana seedling lysate
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9907, RMSE = 0.0336
2025-12-10 12:35:41 - SigmoidF

c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: divide by zero encountered in divide
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.8744, RMSE = 0.0997
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.319944, a = 2338.444725, b = 60.652766
2025-12-10 12:35:41 - MeltomeAtlasHandler - INFO - Chunk progress : 9 / 25, pid : Q9LK72, specie : Arabidopsis thaliana seedling lysate
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9828, RMSE = 0.0551
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.144246, a = 1301.472266, b = 26.599504
2025-12-10 12:35:41 - MeltomeAtlasHandler - INFO - Chunk progress : 10 / 25, pid : P24469, specie : Bacillus subtilis_168_lysate_R1
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.6765, RMSE = 0.1095
2025-12-10 12:35:41 - SigmoidFitter - DEBUG - Fitted parameters: pl = 1.52623

c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: overflow encountered in exp
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


2025-12-10 12:35:51 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 12:35:51 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9914, RMSE = 0.0364
2025-12-10 12:35:51 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.041063, a = 2064.618001, b = 26.916772
2025-12-10 12:35:51 - MeltomeAtlasHandler - INFO - Chunk progress : 12 / 25, pid : Q6L135, specie : Picrophilus torridus DSM9790 lysate
2025-12-10 12:35:51 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 12:35:51 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9908, RMSE = 0.0336
2025-12-10 12:35:51 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.009986, a = 1146.812016, b = 15.754494
2025-12-10 12:35:51 - MeltomeAtlasHandler - INFO - Chunk progress : 13 / 25, pid : Q6L1S5, specie : Picrophilus torridus DSM9790 lysate
2025-12-10 12:35:51 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 12:35:51 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9882, RMSE = 

In [24]:
pid, runName = results.loc[0][['pid', 'runName']]
print(pid, runName)

Q9ZV56 Arabidopsis thaliana seedling lysate


In [23]:
data_handler.data[(data_handler.data.uniprotAccession == pid) & (data_handler.data.runName == runName)]

,proteinId,uniprotAccession,runName,meltingPoint,meltingBehaviour,sequence,quantNormMeltingPoint
37809,Q9ZV56_CCT1,Q9ZV56,Arabidopsis thaliana seedling lysate,45.937513,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MSNVIGDRTEDGLSTAAAASGSTAVQSSPPTDRPVRVYADGIYDLF...,NaN


In [14]:
results[results.isna().any(axis=1)]

,pid,runName,pl,a,b,rmse,r_squared,tm_pred,tm_flip
2,O80653,Arabidopsis thaliana seedling lysate,1.584738,12278.741270,184.367640,0.1705,0.6064,0.00,NaN
3,Q9C8E6,Arabidopsis thaliana seedling lysate,0.758364,8635.097088,190.735242,0.0871,0.6579,55.81,NaN
4,Q9CA28,Arabidopsis thaliana seedling lysate,0.726809,3362.087545,111.536905,0.2327,0.1349,44.59,NaN
7,Q38946,Arabidopsis thaliana seedling lysate,0.682479,9727.274079,194.453647,0.1309,0.4914,61.50,NaN
10,P24469,Bacillus subtilis_168_lysate_R1,1.526235,3146.749194,56.608684,0.1095,0.6765,0.00,NaN
11,O34714,Bacillus subtilis_168_lysate_R1,354.642875,1182.026173,12.794210,0.1053,0.4803,0.00,NaN
20,G5EGM4,Caenorhabditis elegans lysate,0.879966,9051.691975,318.640906,0.1444,0.0988,31.95,NaN
23,G5EFM6,Caenorhabditis elegans lysate,2.535636,2199.472596,46.877294,0.1484,0.9585,0.00,NaN
25,Q9U1T6,Caenorhabditis elegans lysate,NaN,NaN,NaN,NaN,NaN,0.00,NaN
27,Q2HQL2,Caenorhabditis elegans lysate,0.618363,730.817824,18.778930,0.0582,0.9093,100.00,NaN
